In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
dataset = load_dataset("SetFit/tweet_eval_stance", "stance_abortion")

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/126 [00:00<?, ?B/s]

tweet_eval_stance.py:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

stance_abortion/train/0000.parquet:   0%|          | 0.00/44.2k [00:00<?, ?B/s]

stance_abortion/test/0000.parquet:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/587 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/346 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 587
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 346
    })
})


In [3]:
train_data = dataset['train']
test_data = dataset['test']

In [4]:
import pandas as pd

df_train = pd.DataFrame(train_data)
print(df_train.head())

                                                text  label label_text
0  we remind ourselves that love means to be will...      1    against
1  @user @user and most Islanders have different ...      0       none
2  Life is #precious & so are babies, mothers, & ...      1    against
3  @user too many people are taking this to serio...      0       none
4  Dude i won a #freeshirt from @user ! I never w...      0       none


In [5]:
df_test = pd.DataFrame(test_data)
df_test

,text,label,label_text
0,Need a ProLife R.E. Agent? - Support a ProLife...,1,against
1,Where is the childcare program @user which you...,1,against
2,I get several requests with petitions to save ...,1,against
3,"we must always see others as Christ sees us,we...",1,against
4,PRAYERS FOR BABIES Urgent prayer one in Lexing...,1,against
...,...,...,...
341,It's time to be a #voice for the #voiceless. S...,1,against
342,BRAVO TO BOTH OF YOU BEAUTIFULLY EXPLAINED HOL...,2,favor
343,@user - I guess this means you're happy for me...,1,against
344,"My body, my life. You fuck it up in a way I'm ...",2,favor


In [6]:
df_train['label'].unique()

array([1, 0, 2])

In [7]:
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
test_encodings = test_data.map(preprocess_function, batched=True)
test_encodings.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/346 [00:00<?, ? examples/s]

In [11]:
def predict(model, dataset):
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for batch in dataset:
            inputs = {
                "input_ids": batch["input_ids"].unsqueeze(0),
                "attention_mask": batch["attention_mask"].unsqueeze(0),
            }
            outputs = model(**inputs)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.append(pred[0])
            labels.append(batch["label"])
    return np.array(predictions), np.array(labels)
predictions, true_labels = predict(model, test_encodings)
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average="weighted")

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.1676


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Точность без обучения получилась небольшой

In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [12]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

train_encodings = train_data.map(preprocess_function, batched=True)
test_encodings = test_data.map(preprocess_function, batched=True)

train_encodings.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_encodings.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate()
print("Evaluation results:", results)


Map:   0%|          | 0/587 [00:00<?, ? examples/s]

Map:   0%|          | 0/346 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.035100,0.861633,0.627168,0.714797,0.627168,0.582009
2,0.722800,0.748524,0.702312,0.740141,0.702312,0.711360
3,0.421900,0.801049,0.710983,0.765641,0.710983,0.722204


Evaluation results: {'eval_loss': 0.7485244870185852, 'eval_accuracy': 0.7023121387283237, 'eval_precision': 0.7401411819433458, 'eval_recall': 0.7023121387283237, 'eval_f1': 0.7113598892759231, 'eval_runtime': 2.5111, 'eval_samples_per_second': 137.789, 'eval_steps_per_second': 17.522, 'epoch': 3.0}


Точность модели увеличилась